In [1]:
import numpy as np
import torch

/home/renan/regul/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

In [13]:
x_data.shape, x_data.dtype, x_data.device

(torch.Size([2, 2]), torch.int64, device(type='cpu'))

In [35]:
x_data @ torch.ones_like(x_data)
x_data * torch.ones_like(x_data)
x_data @ torch.eye(2,2, dtype=torch.long)
x_data * torch.eye(2,2, dtype=torch.long)

tensor([[1, 0],
        [0, 4]])

In [32]:
x_data.dtype

torch.int64

In [36]:
x_data.numpy()

array([[1, 2],
       [3, 4]])

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# pyG Dataset

In [34]:
import networkx as nx
import torch

In [65]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_networkx, index_to_mask, mask_to_index

In [104]:
G = nx.DiGraph()
G.add_edges_from([
    (0, 2),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 5),
    (4, 6)
])

In [46]:
torch.tensor([2])
index_to_mask(torch.tensor([2]))

tensor([False, False,  True])

In [49]:
d = from_networkx(G)
d.train_mask = index_to_mask(torch.tensor([2]), size=d.num_nodes)
d2 = from_networkx(G)
d2.train_mask = index_to_mask(torch.tensor([3]), size=d.num_nodes)

In [52]:
loader = DataLoader([d, d2, d, d2], batch_size=3, shuffle=True)

In [58]:
for data in loader:
    print(data.train_mask)

tensor([False, False, False,  True, False, False, False, False, False,  True,
        False, False, False, False, False, False, False,  True, False, False,
        False])
tensor([False, False,  True, False, False, False, False])


In [64]:
out = torch.rand(7)
out[loader[0].train_mask]

tensor([0.9619])

In [71]:
d.node_stores

[{'edge_index': tensor([[0, 1, 2, 3, 4, 4],
         [1, 3, 1, 4, 5, 6]]), 'num_nodes': 7, 'train_mask': tensor([False, False,  True, False, False, False, False])}]

In [81]:
from torch_geometric.utils import to_networkx
G2 = to_networkx(d)
# nx.draw_networkx(G2)

In [80]:
# nx.draw_networkx(G)

In [108]:
G = nx.DiGraph()
G.add_edges_from([
    ('A', 'C'),
    ('B', 'C'),
    ('C', 'D'),
    ('D', 'E'),
    ('D', 'F'),
])

attr = {node:i for i,node in enumerate(G.nodes)}
nx.set_node_attributes(G, attr, name='toto')
G = nx.convert_node_labels_to_integers(G, label_attribute='name')

In [109]:
d = from_networkx(G)
d.node_stores

[{'edge_index': tensor([[0, 1, 2, 3, 3],
         [1, 3, 1, 4, 5]]), 'toto': tensor([0, 1, 2, 3, 4, 5]), 'name': ['A', 'C', 'B', 'D', 'E', 'F'], 'num_nodes': 6}]

In [112]:
d.name.index('C')

1

In [88]:
d.x